VSTOXX > 대표적인 유럽의 변동성 지수로, VSTOXX 는 유럽의 대표 주가지수인 Euro STOXX의 변동성 지수로 Euro STOXX 50 지수 옵션가격 으로부터 산출되며 Euro Exchange에서 편제･발표한다. 우리나라도 KOSPI 200 지수의 변동성지수인 VKOSPI가 2009년 도입되었다.

In [1]:
import numpy as np
import pandas as pd

**VSTOXX Index Data**

In [2]:
#chapter 16에서 이용한 indices.
url = 'http://www.stoxx.com/download/historical_values/h_vstoxx.txt'
vstoxx_index = pd.read_csv(url,index_col = 0, header = 2,
                          parse_dates = True, dayfirst = True)

In [3]:
vstoxx_index.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4357 entries, 1999-01-04 to 2016-02-12
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V2TX    4357 non-null   float64
 1   V6I1    3906 non-null   float64
 2   V6I2    4357 non-null   float64
 3   V6I3    4296 non-null   float64
 4   V6I4    4357 non-null   float64
 5   V6I5    4357 non-null   float64
 6   V6I6    4340 non-null   float64
 7   V6I7    4357 non-null   float64
 8   V6I8    4343 non-null   float64
dtypes: float64(9)
memory usage: 340.4 KB


In [4]:
#2014의 1분기만 필요하므로 나머지 삭제
vstoxx_index = vstoxx_index[('2013/12/31'< vstoxx_index.index)&(vstoxx_index.index < '2014/4/1')]

In [5]:
np.round(vstoxx_index.tail(),2)

,V2TX,V6I1,V6I2,V6I3,V6I4,V6I5,V6I6,V6I7,V6I8
Date,,,,,,,,,
2014-03-25,18.26,18.23,18.31,19.04,19.84,20.31,18.11,20.83,21.20
2014-03-26,17.59,17.48,17.70,18.45,19.42,20.00,20.26,20.45,20.86
2014-03-27,17.64,17.50,17.76,18.62,19.49,20.05,20.11,20.49,20.94
2014-03-28,17.03,16.68,17.29,18.33,19.30,19.83,20.14,20.38,20.82
2014-03-31,17.66,17.61,17.69,18.57,19.43,20.04,19.98,20.44,20.90


**VSTOXX Futures Data**\
vstoxx index data는 공적으로 사용 불가.

In [6]:
vstoxx_futures = pd.read_excel('/Users/dpwl0/OneDrive/바탕화면/Finance/vstoxx_march_2014.xlsx',
                               'vstoxx_futures') #source는 github py4fi에서 받았고, 저장해둔위치로 코드 변경
vstoxx_futures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   A_DATE                     504 non-null    datetime64[ns]
 1   A_EXP_YEAR                 504 non-null    int64         
 2   A_EXP_MONTH                504 non-null    int64         
 3   A_CALL_PUT_FLAG            504 non-null    object        
 4   A_EXERCISE_PRICE           504 non-null    int64         
 5   A_SETTLEMENT_PRICE_SCALED  504 non-null    int64         
 6   A_PRODUCT_ID               504 non-null    object        
 7   SETTLE                     504 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 31.6+ KB


In [7]:
#필요없는 행 제거
del vstoxx_futures['A_SETTLEMENT_PRICE_SCALED']
del vstoxx_futures['A_CALL_PUT_FLAG']
del vstoxx_futures['A_EXERCISE_PRICE']
del vstoxx_futures['A_PRODUCT_ID']

In [8]:
columns = ['DATE','EXP_YEAR','EXP_MONTH','PRICE']
vstoxx_futures.columns = columns

In [9]:
import datetime as dt
import calendar

In [10]:
def third_friday(date):
    day = 21-(calendar.weekday(date.year,date.month,1)+2)%7
    return dt.datetime(date.year,date.month,day)

In [11]:
set(vstoxx_futures['EXP_MONTH'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

In [12]:
#2014/4/18 독일 공휴일이지만 그와 상관없이 분석에 나옴
third_fridays = {}
for month in set(vstoxx_futures['EXP_MONTH']):
    third_fridays[month] = third_friday(dt.datetime(2014, month, 1))

In [13]:
third_fridays

{1: datetime.datetime(2014, 1, 17, 0, 0),
 2: datetime.datetime(2014, 2, 21, 0, 0),
 3: datetime.datetime(2014, 3, 21, 0, 0),
 4: datetime.datetime(2014, 4, 18, 0, 0),
 5: datetime.datetime(2014, 5, 16, 0, 0),
 6: datetime.datetime(2014, 6, 20, 0, 0),
 7: datetime.datetime(2014, 7, 18, 0, 0),
 8: datetime.datetime(2014, 8, 15, 0, 0),
 9: datetime.datetime(2014, 9, 19, 0, 0),
 10: datetime.datetime(2014, 10, 17, 0, 0),
 11: datetime.datetime(2014, 11, 21, 0, 0)}

In [14]:
tf = lambda x: third_fridays[x]
vstoxx_futures['MATURITY'] = vstoxx_futures['EXP_MONTH'].apply(tf)

In [15]:
vstoxx_futures.tail()

,DATE,EXP_YEAR,EXP_MONTH,PRICE,MATURITY
499,2014-03-31,2014,7,20.40,2014-07-18
500,2014-03-31,2014,8,20.70,2014-08-15
501,2014-03-31,2014,9,20.95,2014-09-19
502,2014-03-31,2014,10,21.05,2014-10-17
503,2014-03-31,2014,11,21.25,2014-11-21


**VSTOXX Options Data**

In [16]:
vstoxx_options = pd.read_excel('/Users/dpwl0/OneDrive/바탕화면/Finance/vstoxx_march_2014.xlsx',
                              'vstoxx_options')

In [17]:
vstoxx_options.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46960 entries, 0 to 46959
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   A_DATE                     46960 non-null  datetime64[ns]
 1   A_EXP_YEAR                 46960 non-null  int64         
 2   A_EXP_MONTH                46960 non-null  int64         
 3   A_CALL_PUT_FLAG            46960 non-null  object        
 4   A_EXERCISE_PRICE           46960 non-null  int64         
 5   A_SETTLEMENT_PRICE_SCALED  46960 non-null  int64         
 6   A_PRODUCT_ID               46960 non-null  object        
 7   SETTLE                     46960 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 2.9+ MB


In [18]:
del vstoxx_options['A_SETTLEMENT_PRICE_SCALED']
del vstoxx_options['A_PRODUCT_ID']

In [20]:
columns = ['DATE','EXP_YEAR','EXP_MONTH','TYPE','STRIKE','PRICE']
vstoxx_options.columns = columns

In [21]:
vstoxx_options['MATURITY'] = vstoxx_options['EXP_MONTH'].apply(tf)

In [22]:
vstoxx_options.head()

,DATE,EXP_YEAR,EXP_MONTH,TYPE,STRIKE,PRICE,MATURITY
0,2014-01-02,2014,1,C,1000,7.95,2014-01-17
1,2014-01-02,2014,1,C,1500,3.05,2014-01-17
2,2014-01-02,2014,1,C,1600,2.20,2014-01-17
3,2014-01-02,2014,1,C,1700,1.60,2014-01-17
4,2014-01-02,2014,1,C,1800,1.15,2014-01-17


In [23]:
vstoxx_options['STRIKE'] = vstoxx_options['STRIKE']/100.

In [24]:
save = False
if save is True:
    import warnings
    warnings.simplefilter('ignore')
    h5 = pd.HDFStore('/Users/dpwl0/OneDrive/바탕화면/Finance/vstoxx_march_2014.h5',
                    complevel = 9, complib = 'blosc')
    h5['vstoxx_index'] = vstoxx_index
    h5['vstoxx_futures'] = vstoxx_futures
    h5['vstoxx_options'] = vstoxx_options